In [ ]:
# Run with Kernel: gemini-chat-env

In [9]:
# ==============================================================================
# Imports
# ==============================================================================

import os
import pickle
import time
import random
from typing import List, Dict, Any
from google import genai
from google.genai import types

In [ ]:
# ==============================================================================
# API key
# ==============================================================================
def read_key_from_env_file(filename: str = "key.env", var_name: str = "GEMINI_API_KEY") -> str:
    p = Path.cwd() / filename  # current working directory (your notebook folder)
    if not p.exists():
        raise FileNotFoundError(f"'{filename}' not found in {Path.cwd()}")
    for line in p.read_text(encoding="utf-8").splitlines():
        line = line.strip()
        if not line or line.startswith("#"):
            continue
        if "=" in line:
            k, v = line.split("=", 1)
            if k.strip() == var_name:
                return v.strip().strip('"').strip("'")
    raise RuntimeError(f"{var_name} not found in {filename}")

API_KEY = read_key_from_env_file() 


In [3]:
# ==============================================================================
# Model selection
# ==============================================================================
EMBED_MODEL = "gemini-embedding-001"


In [4]:
# ==============================================================================
# Config
# ==============================================================================
INPUT_PKL  = "repackaged_transcript_data.pkl"
OUTPUT_PKL = "transcript_embeddings.pkl"

CUSTOMERS_TO_PROCESS = 50
SLEEP_SECONDS = 1.0       # base delay after successful call
MAX_RETRIES   = 6         # for 503/429/timeouts
BACKOFF_BASE  = 1.6       # exponential backoff base
BACKOFF_CAP   = 30.0      # max sleep per retry



In [12]:
# ==============================================================================
# Helpers
# ==============================================================================
def atomic_pickle(obj, path: str):
    tmp = path + ".tmp"
    with open(tmp, "wb") as f:
        pickle.dump(obj, f)
    os.replace(tmp, path)

def load_input() -> List[List[str]]:
    with open(INPUT_PKL, "rb") as f:
        return pickle.load(f)

def load_existing() -> List[Dict[str, Any]]:
    if not os.path.exists(OUTPUT_PKL):
        return []
    with open(OUTPUT_PKL, "rb") as f:
        return pickle.load(f)


def embed_one(client: genai.Client, text: str) -> List[float]:
    """
    Embed a transcript (document) for RAG using RETRIEVAL_DOCUMENT.
    Returns a 3072-d vector (list[float]) to match your stored shape.
    """
    cfg = types.EmbedContentConfig(
        task_type="RETRIEVAL_DOCUMENT",
        output_dimensionality=3072,  # keep consistent with your store
    )

    last_err = None
    for attempt in range(MAX_RETRIES):
        try:
            r = client.models.embed_content(
                model=EMBED_MODEL,
                contents=text,
                config=cfg,
            )
            return r.embeddings[0].values  # 3072-d vector
        except Exception as e:
            msg = str(e)
            if "FreeTier" in msg and "limit: 0" in msg:
                raise RuntimeError(
                    "Embedding FreeTier is 0/Unavailable. Enable billing or switch project."
                )
            if any(s in msg for s in ("503", "UNAVAILABLE", "overloaded", "429", "timeout", "Deadline")):
                sleep_s = min(BACKOFF_BASE ** attempt + random.random(), BACKOFF_CAP)
                time.sleep(sleep_s)
                last_err = e
                continue
            raise
    raise last_err

In [13]:
# ==============================================================================
# Main
# ==============================================================================
def main():
    # Client
    client = genai.Client(api_key=API_KEY)

    # Load input
    try:
        customer_data = load_input()
    except FileNotFoundError:
        raise FileNotFoundError(f"Could not find '{INPUT_PKL}'. Make sure the file exists.")
    print(f"Loaded {len(customer_data)} customer records from '{INPUT_PKL}'.")

    # Resume support: load existing output and build set of done ids
    processed = load_existing()
    done_ids = {rec["customer_id"] for rec in processed}
    total = min(CUSTOMERS_TO_PROCESS, len(customer_data))
    print(f"🚀 Starting (resume) for {total} customers; already done: {len(done_ids)}")

    for i, transcripts in enumerate(customer_data[:total]):
        if i in done_ids:
            print(f"↩️  Skipping customer {i+1}/{total} (already processed)")
            continue

        try:
            embs = []
            for t in transcripts:
                embs.append(embed_one(client, t))
                time.sleep(SLEEP_SECONDS)

            rec = {"customer_id": i, "transcripts": transcripts, "embeddings": embs}
            processed.append(rec)

            # Save after each customer (atomic) so we can safely resume later
            atomic_pickle(processed, OUTPUT_PKL)
            print(f"✅ Processed customer {i+1}/{total}")

        except Exception as e:
            print(f"⛔ Stopping on customer {i+1}. Details:\n{e}")
            break

    print(f"\nDone. Saved {len(processed)} customers to '{OUTPUT_PKL}'.")

if __name__ == "__main__":
    main()

Loaded 50 customer records from 'repackaged_transcript_data.pkl'.
🚀 Starting (resume) for 50 customers; already done: 0
✅ Processed customer 1/50
✅ Processed customer 2/50
✅ Processed customer 3/50
✅ Processed customer 4/50
✅ Processed customer 5/50
✅ Processed customer 6/50
✅ Processed customer 7/50
✅ Processed customer 8/50
✅ Processed customer 9/50
✅ Processed customer 10/50
✅ Processed customer 11/50
✅ Processed customer 12/50
✅ Processed customer 13/50
✅ Processed customer 14/50
✅ Processed customer 15/50
✅ Processed customer 16/50
✅ Processed customer 17/50
✅ Processed customer 18/50
✅ Processed customer 19/50
✅ Processed customer 20/50
✅ Processed customer 21/50
✅ Processed customer 22/50
✅ Processed customer 23/50
✅ Processed customer 24/50
✅ Processed customer 25/50
✅ Processed customer 26/50
✅ Processed customer 27/50
✅ Processed customer 28/50
✅ Processed customer 29/50
✅ Processed customer 30/50
✅ Processed customer 31/50
✅ Processed customer 32/50
✅ Processed customer 33/5

In [14]:
# ==============================================================================
# Check file contents
# ==============================================================================
import statistics
from typing import List, Dict, Any
import numpy as np

INPUT_PKL = "transcript_embeddings.pkl"

def load_pickle(path: str) -> List[Dict[str, Any]]:
    with open(path, "rb") as f:
        return pickle.load(f)

def cosine_matrix(X: np.ndarray) -> np.ndarray:
    # X: (n, d)
    norms = np.linalg.norm(X, axis=1, keepdims=True) + 1e-12
    Xn = X / norms
    return Xn @ Xn.T  # (n, n)

def main():
    data = load_pickle(INPUT_PKL)
    n_customers = len(data)
    print(f"✅ Loaded {n_customers} customer records from '{INPUT_PKL}'.")

    # Validate structure and collect stats
    bad_struct = []
    dims = []
    norms = []
    lens_chars = []

    for rec_idx, rec in enumerate(data):
        if not isinstance(rec, dict):
            bad_struct.append((rec_idx, "record not dict"))
            continue

        cid = rec.get("customer_id")
        transcripts = rec.get("transcripts")
        embeddings = rec.get("embeddings")

        if not isinstance(transcripts, list) or len(transcripts) != 4:
            bad_struct.append((cid, "transcripts != 4"))
        if not isinstance(embeddings, list) or len(embeddings) != 4:
            bad_struct.append((cid, "embeddings != 4"))

        # Stats
        for t in (transcripts or []):
            if isinstance(t, str):
                lens_chars.append(len(t))

        for v in (embeddings or []):
            if isinstance(v, (list, tuple, np.ndarray)):
                dims.append(len(v))
                norms.append(float(np.linalg.norm(np.asarray(v, dtype=np.float32))))

    if bad_struct:
        print(f"⚠️ Found {len(bad_struct)} structural issues (showing up to 5): {bad_struct[:5]}")
    else:
        print("✅ All records have 4 transcripts and 4 embeddings.")

    if dims:
        unique_dims = sorted(set(dims))
        print(f"📐 Embedding dimension(s) present: {unique_dims} (expected single value, typically 3072)")
    if norms:
        print(f"📊 Embedding L2-norm — mean: {statistics.mean(norms):.3f}, "
              f"min: {min(norms):.3f}, max: {max(norms):.3f}")

    if lens_chars:
        print(f"📝 Transcript length (chars) — mean: {statistics.mean(lens_chars):.1f}, "
              f"median: {statistics.median(lens_chars):.0f}, min: {min(lens_chars)}, max: {max(lens_chars)}")

    # Quick per-customer similarity glance for the first 3 customers
    print("\n🔎 Intra-customer cosine similarity matrices (first up to 3 customers):")
    for rec in data[:3]:
        cid = rec["customer_id"]
        E = np.array(rec["embeddings"], dtype=np.float32)  # (4, d)
        C = cosine_matrix(E)
        np.set_printoptions(precision=3, suppress=True)
        print(f"\nCustomer {cid} (4x4 cosine):\n{C}")

    # Show one example record (truncated text) to confirm alignment
    if n_customers:
        r0 = data[0]
        print("\n🧪 Sample record 0 overview:")
        for k in ("customer_id",):
            print(f"  {k}: {r0[k]}")
        for idx, t in enumerate(r0["transcripts"]):
            excerpt = (t[:180] + "…") if len(t) > 180 else t
            print(f"  transcript[{idx}]: {len(t)} chars | {excerpt}")
        for idx, v in enumerate(r0["embeddings"]):
            print(f"  embedding[{idx}]: dim={len(v)}  norm={np.linalg.norm(np.asarray(v, dtype=np.float32)):.3f}")

if __name__ == "__main__":
    main()

✅ Loaded 50 customer records from 'transcript_embeddings.pkl'.
✅ All records have 4 transcripts and 4 embeddings.
📐 Embedding dimension(s) present: [3072] (expected single value, typically 3072)
📊 Embedding L2-norm — mean: 1.000, min: 1.000, max: 1.000
📝 Transcript length (chars) — mean: 2334.2, median: 2245, min: 1899, max: 3426

🔎 Intra-customer cosine similarity matrices (first up to 3 customers):

Customer 0 (4x4 cosine):
[[1.    0.8   0.788 0.793]
 [0.8   1.    0.753 0.737]
 [0.788 0.753 1.    0.853]
 [0.793 0.737 0.853 1.   ]]

Customer 1 (4x4 cosine):
[[1.    0.781 0.784 0.779]
 [0.781 1.    0.802 0.799]
 [0.784 0.802 1.    0.906]
 [0.779 0.799 0.906 1.   ]]

Customer 2 (4x4 cosine):
[[1.    0.82  0.804 0.806]
 [0.82  1.    0.821 0.808]
 [0.804 0.821 1.    0.874]
 [0.806 0.808 0.874 1.   ]]

🧪 Sample record 0 overview:
  customer_id: 0
  transcript[0]: 2006 chars | Thank you for calling [ORGANIZATION] [ORGANIZATION] [ORGANIZATION], [ORGANIZATION]. This is [PERSON_NAME] speaking.